In [ ]:
# Importing necessary libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
!pip install torchsummary
from torchsummary import summary
# Setting the device to gpu or cpu based on availability
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Train data transformations
train_transforms = transforms.Compose([
    transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
    transforms.Resize((28, 28)),
    transforms.RandomRotation((-15., 15.), fill=0),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

# Test data transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

In [ ]:
# Getting the train and test data
train_data = datasets.MNIST('../data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('../data', train=False, download=True, transform=test_transforms)

In [ ]:
batch_size = 128

kwargs = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 2, 'pin_memory': True}

test_loader = torch.utils.data.DataLoader(test_data, **kwargs)
train_loader = torch.utils.data.DataLoader(train_data, **kwargs)

In [ ]:
class Net(nn.Module):
  def __init__(self):
      super(Net, self).__init__()
      # Extracting 16 features using 3x3 kernel but keeping size same
      self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #rin = 1 rout= 3
      # Performing batchNormalization
      self.bn1 = nn.BatchNorm2d(16)
      # Performing maxPooling assuming 1st level of features are extracted
      self.pool1 = nn.MaxPool2d(2, 2); #rin = 3 rout= 4
      # Avoiding overfitting
      self.dropout1 = nn.Dropout(0.10);
      # Extracting 2nd level of features
      self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #rin = 4 rout= 8
      # Performing batchNormalization
      self.bn2 = nn.BatchNorm2d(32)
      # Performing maxPooling assuming 2nd level of features are extracted
      self.pool2 = nn.MaxPool2d(2, 2); #rin = 8 rout= 10
      # Avoiding overfitting
      self.dropout2 = nn.Dropout(0.10);
      # Performing fully connected but maintaining spatial information
      self.conv3 = nn.Conv2d(32, 64, 1) #rin = 10 rout= 10
      self.bn3 = nn.BatchNorm2d(64)
      # Extract the important information and increase receptive field
      self.pool3 = nn.MaxPool2d(2, 2); #rin = 10 rout = 14
      # Getting info for 10 classes
      self.conv4 = nn.Conv2d(64, 10, 3) #rin = 14 rout= 30
      
  def forward(self, x):
    x = self.pool1(self.bn1(F.relu(self.conv1(x))))
    x = self.dropout1(x)
    x = self.pool2(self.bn2(F.relu(self.conv2(x))))
    x = self.dropout2(x)
    x = self.pool3(self.bn3(F.relu(self.conv3(x))))
    x = self.conv4(x)
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=1)


In [ ]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
           Dropout-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           4,640
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 64, 7, 7]           2,112
      BatchNorm2d-10             [-1, 64, 7, 7]             128
        MaxPool2d-11             [-1, 64, 3, 3]               0
           Conv2d-12             [-1, 10, 1, 1]           5,770
Total params: 12,906
Trainable params: 12,906
Non-trainable params: 0
---------------------------------

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.05793764069676399 batch_id=468: 100%|██████████| 469/469 [01:04<00:00,  7.30it/s]



Test set: Average loss: 0.0561, Accuracy: 9805/10000 (98.05%)

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.07863428443670273 batch_id=468: 100%|██████████| 469/469 [01:03<00:00,  7.35it/s]



Test set: Average loss: 0.0334, Accuracy: 9890/10000 (98.90%)

Adjusting learning rate of group 0 to 1.0000e-02.


...


Test set: Average loss: 0.0200, Accuracy: 9933/10000 (99.33%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.0797007605433464 batch_id=468: 100%|██████████| 469/469 [01:04<00:00,  7.29it/s]



Test set: Average loss: 0.0200, Accuracy: 9930/10000 (99.30%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.007199639454483986 batch_id=468: 100%|██████████| 469/469 [01:04<00:00,  7.26it/s]



Test set: Average loss: 0.0199, Accuracy: 9933/10000 (99.33%)

Adjusting learning rate of group 0 to 1.0000e-03.
